In [3]:
#Libraries

In [27]:
import requests
from bs4 import BeautifulSoup as bs
import time
import random
import pandas as pd
import json
import matplotlib.pyplot as plt
import lxml

In [11]:
headers = {
    "accept": 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    "accept-encoding": 'gzip, deflate, br',
    "accept-language": 'es-ES,es;q=0.9,en;q=0.8',
    "cache-control": 'max-age=0',
    "dnt": "1",
    "sec-ch-ua": '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    "sec-ch-ua-mobile": '?0',
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": 'document',
    "sec-fetch-mode": 'navigate',
    "sec-fetch-site": 'same-origin',
    "sec-fetch-user": '?1',
    "upgrade-insecure-requests": '1',
    "user-agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36'
}

In [15]:
def request_piso(id_inmueble, headers=""):
    url = f"https://www.idealista.com/inmueble/{id_inmueble}/"
    r = requests.get(url, headers=headers)
    time.sleep(random.randint(1,3)*random.random())
    return r

In [16]:
r = request_piso("95931901", headers)

In [17]:
r

<Response [200]>

In [29]:
soup = bs(r.text, "html.parser")

In [30]:
def info_inmueble(soup):
    
    scripts = soup.find_all("script")
    
    for script in scripts:
        if "var config" in str(script):
            break
    latitude = float(str(script).split("latitude")[1].split(",")[0].split()[1].replace("'", ""))        
    longitude = float(str(script).split("latitude")[1].split(",")[1].strip().split()[1].replace("'", ""))
    
    scripts = soup.find_all("script")
    
    for script in scripts:
        if "var utag_data" in str(script):
            break
    
    data = json.loads(str(script).split("var utag_data =")[1].split(";")[0].strip())
    
    titulo = soup.find("h1").text.strip()
    localizacion = soup.find("span",{"class":"main-info__title-block"}).find("span").text.strip()
    price = data["ad"]["price"]
    energy_certification = data["ad"]["energyCertification"]["type"]
    basic_characteristics = data["ad"]["characteristics"]
    room_number = data["ad"]["characteristics"]["roomNumber"]
    bath_number = data["ad"]["characteristics"]["bathNumber"]
    
    try:
        has_garden = data["ad"]["characteristics"]["hasGarden"]
    except:
        has_garden = None
        
    try:
        has_terrace = data["ad"]["characteristics"]["hasTerrace"]
    except:
        has_terrace = None
        
    try:
        has_parking = data["ad"]["characteristics"]["hasParking"]
    except:
        has_parking = None
        
    try:
        has_swimmingpool = data["ad"]["characteristics"]["hasSwimmingPool"]
    except:
        has_swimmingpool = None
        
    try:
        has_lift = data["ad"]["characteristics"]["hasLift"]
    except:
        has_lift = None
        
    constructed_area = data["ad"]["characteristics"]["constructedArea"]
    is_new_development = data["ad"]["condition"]["isNewDevelopment"]
    is_needs_renovating = data["ad"]["condition"]["isNeedsRenovating"]
    is_goog_condition = data["ad"]["condition"]["isGoodCondition"]
    
    lista = [titulo, localizacion, latitude, longitude, price, energy_certification, basic_characteristics,
            room_number, bath_number,has_garden, has_terrace, has_parking, has_swimmingpool, has_lift, constructed_area,
            is_new_development, is_needs_renovating, is_goog_condition]
    
    time.sleep(random.randint(1,3)*random.random())
    return lista

In [31]:
info_inmueble(soup)

['Casa o chalet independiente en venta en calle Poseidón',
 'Churriana-El Pizarrillo-La Noria-Guadalsol, Málaga',
 36.6637191,
 -4.5038972,
 '640000',
 'c',
 {'roomNumber': '4',
  'bathNumber': '2',
  'hasParking': '1',
  'hasGarden': '1',
  'hasSwimmingPool': '1',
  'hasTerrace': '1',
  'constructedArea': '310'},
 '4',
 '2',
 '1',
 '1',
 '1',
 '1',
 None,
 '310',
 '0',
 '0',
 '1']

In [12]:
#identificación de IDS

In [34]:
busqueda = "alhaurin de la torre malaga"
busqueda = busqueda.replace(" ","_")

x = 1
ids = []

while True:
    url = f"https://www.idealista.com/buscar/venta-viviendas/{busqueda}/pagina-{x}.htm"
    
    r = requests.get(url, headers = headers)
    soup = bs(r.text, "html.parser")
    
    pagina_actual = int(soup.find("main", {"class":"listing-items"}).find("div", {"class":"pagination"}).find("li", {"class":"selected"}).text)
    
    if x == pagina_actual:
        articles = soup.find("main", {"class":"listing-items"}).find_all("article")
    
    else:
        break
        
    x = x+1
        
    for article in articles:
        id_inmueble = article.get("data-adid")
        
        ids.append(id_inmueble)
        
    time.sleep(random.randint(1,3)*random.random())

In [35]:
len(ids)

925

In [37]:
#voy a hacer un csv con la columna de ids sacada.
columns_ids = ["id"]
df_ids = pd.DataFrame(ids, columns= columns_ids)

df_ids.to_csv('ids_alhaurin-de-la-torre-malaga.csv')

In [40]:
df = pd.read_csv('ids_alhaurin-de-la-torre-malaga.csv')

In [41]:
df

,Unnamed: 0,id
0,0,98960708
1,1,99113720
2,2,98242505
3,3,100104816
4,4,99840698
...,...,...
920,920,40122591
921,921,98704004
922,922,97673959
923,923,26878655


In [18]:
list_ids = df["id"].tolist()

In [19]:
list_ids

[98811885,
 95931901,
 98750560,
 98942399,
 97949995,
 95499926,
 99338182,
 98719316,
 98822161,
 99045241,
 98960009,
 96318991,
 99129858,
 97702649,
 99417058,
 99343448,
 99213688,
 97366293,
 98655741,
 97009508,
 96966283,
 96264226,
 99267735,
 99288860,
 99288592,
 98893481,
 98873419,
 98557538,
 97527823,
 97143561,
 96459404,
 98801384,
 94804199,
 99026825,
 97210242,
 98124262,
 97634541,
 97124225,
 99288393,
 98281131,
 97431795,
 97008087,
 99199774,
 99101562,
 98345112,
 97283582,
 92690638,
 86709875,
 95767352,
 94620374,
 91773542,
 98616786,
 97317917,
 97597379,
 99375174,
 99285180,
 98961792,
 98076552,
 98690124,
 95011891,
 89127955,
 96830639,
 81246662,
 96481816,
 81395404,
 98635787,
 99124298,
 98948359,
 98483847,
 99282287,
 97741878,
 99354548,
 99309774,
 98904108,
 36820701,
 98490064,
 99362718,
 98606809,
 97434759,
 99261963,
 99212700,
 98980672,
 98504063,
 99339984,
 98951616,
 98735583,
 87347187,
 98239236,
 99286472,
 89664288,
 87381531,

In [43]:
lista_info_inmuebles = []

for id_inmueble in ids:
    r = request_piso(id_inmueble, headers)
    soup = bs(r.text, "html.parser")
    info_inmueble1 = info_inmueble(soup)
    lista_info_inmuebles.append(info_inmueble1)
    time.sleep(random.randint(1,3)*random.random())

print(id_inmueble)

IndexError: list index out of range

In [ ]:
# Huelin 116 id, '88341181'

In [44]:
len(lista_info_inmuebles)

159

In [46]:
lista_info_inmuebles

[['Chalet pareado en venta en Santa Clara',
  'Capellanía - Retamar, Alhaurín de la Torre',
  36.6631239,
  -4.5312592,
  '499000',
  'd',
  {'roomNumber': '4',
   'bathNumber': '3',
   'hasParking': '1',
   'hasGarden': '1',
   'hasSwimmingPool': '1',
   'hasTerrace': '1',
   'constructedArea': '245'},
  '4',
  '3',
  '1',
  '1',
  '1',
  '1',
  None,
  '245',
  '0',
  '0',
  '1'],
 ['Casa o chalet independiente en venta en calle Manzanilla',
  'Manantiales - Lagar - Cortijo, Alhaurín de la Torre',
  36.6553933,
  -4.5401422,
  '500000',
  'inProcess',
  {'roomNumber': '5',
   'bathNumber': '5',
   'hasParking': '1',
   'hasTerrace': '1',
   'constructedArea': '385'},
  '5',
  '5',
  None,
  '1',
  '1',
  None,
  None,
  '385',
  '0',
  '1',
  '0'],
 ['Casa o chalet independiente en venta en avenida de la Música',
  'Capellanía - Retamar, Alhaurín de la Torre',
  36.66423,
  -4.5468535,
  '629000',
  'inProcess',
  {'roomNumber': '5',
   'bathNumber': '4',
   'hasParking': '1',
   'ha

In [47]:
columns = ["titulo", "localizacion", "latitude", "longitude", "price", "energy_certification", "basic_characteristics",
            "room_number", "bath_number", "has_garden", "has_terrace", "has_parking", "has_swimmingpool", "has_lift", "constructed_area",
            "is_new_development", "is_needs_renovating", "is_good_condition"]


In [48]:
df_inmuebles = pd.DataFrame(lista_info_inmuebles, columns= columns)

In [49]:
df_inmuebles

,titulo,localizacion,latitude,longitude,price,energy_certification,basic_characteristics,room_number,bath_number,has_garden,has_terrace,has_parking,has_swimmingpool,has_lift,constructed_area,is_new_development,is_needs_renovating,is_goog_condition
0,Chalet pareado en venta en Santa Clara,"Capellanía - Retamar, Alhaurín de la Torre",36.663124,-4.531259,499000,d,"{'roomNumber': '4', 'bathNumber': '3', 'hasPar...",4,3,1,1,1,1,None,245,0,0,1
1,Casa o chalet independiente en venta en calle ...,"Manantiales - Lagar - Cortijo, Alhaurín de la ...",36.655393,-4.540142,500000,inProcess,"{'roomNumber': '5', 'bathNumber': '5', 'hasPar...",5,5,None,1,1,None,None,385,0,1,0
2,Casa o chalet independiente en venta en avenid...,"Capellanía - Retamar, Alhaurín de la Torre",36.664230,-4.546854,629000,inProcess,"{'roomNumber': '5', 'bathNumber': '4', 'hasPar...",5,4,1,1,1,1,None,300,0,0,1
3,Casa o chalet independiente en venta en calle ...,"Pinos de Alhaurín, Alhaurín de la Torre",36.651323,-4.589247,560000,e,"{'roomNumber': '6', 'bathNumber': '4', 'hasPar...",6,4,1,1,1,1,None,306,0,0,1
4,Casa o chalet independiente en venta en calle ...,"Pinos de Alhaurín, Alhaurín de la Torre",36.642232,-4.591510,470000,b,"{'roomNumber': '3', 'bathNumber': '3', 'hasPar...",3,3,1,1,1,1,None,200,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Chalet adosado en venta en calle Angola,"Churriana-El Pizarrillo-La Noria-Guadalsol, Má...",36.664299,-4.518129,294000,c,"{'roomNumber': '3', 'bathNumber': '3', 'hasPar...",3,3,0,1,1,1,None,213,0,0,1
155,Casa o chalet independiente en venta en calle ...,"Manantiales - Lagar - Cortijo, Alhaurín de la ...",36.658775,-4.540090,459950,c,"{'roomNumber': '4', 'bathNumber': '3', 'hasPar...",4,3,1,1,1,1,None,142,0,0,1
156,Casa o chalet independiente en venta en calle ...,"Manantiales - Lagar - Cortijo, Alhaurín de la ...",36.661340,-4.534186,837950,c,"{'roomNumber': '6', 'bathNumber': '4', 'hasPar...",6,4,1,1,1,1,None,411,0,0,1
157,Casa o chalet independiente en venta en calle ...,"Platero - El Limón, Alhaurín de la Torre",36.657053,-4.571512,595000,c,"{'roomNumber': '5', 'bathNumber': '3', 'hasPar...",5,3,0,1,1,1,None,400,0,0,1


In [50]:
df_inmuebles.to_csv('info-alhaurin-de-la-torre-malaga.csv')